In [ ]:
!pip install transformers datasets sentencepiece -q
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Load the dataset
file_path = '/content/drive/MyDrive/archive/en-fr_train.csv'  # Update path as needed
df = pd.read_csv(file_path)

# Extract 'en' and 'fr' translations from the 'translation' column
df['en'] = df['translation'].apply(lambda x: eval(x)['en'])
df['fr'] = df['translation'].apply(lambda x: eval(x)['fr'])

# Split the data into training, validation, and test sets
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Save data to CSVs for Hugging Face datasets
train_data.to_csv('train.csv', index=False)
val_data.to_csv('val.csv', index=False)
test_data.to_csv('test.csv', index=False)


In [ ]:
from datasets import load_dataset

# Load the data into Hugging Face datasets
data_files = {
    "train": "train.csv",
    "validation": "val.csv",
    "test": "test.csv"
}
dataset = load_dataset('csv', data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

# Use a pretrained English-French translation model
model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the preprocessing function
def preprocess_function(examples):
    inputs = examples['fr']
    targets = examples['en']

    # Tokenize inputs and targets with padding and truncation
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Map:   0%|          | 0/114376 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/6354 [00:00<?, ? examples/s]

Map:   0%|          | 0/6355 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Load the pretrained translation model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

# Define a data collator to handle padding during batching
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)
# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-2d8f964bed71>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.450000,0.409941
2,0.411500,0.395344
3,0.386700,0.391147


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=21447, training_loss=0.43342361098020155, metrics={'train_runtime': 3726.8956, 'train_samples_per_second': 92.068, 'train_steps_per_second': 5.755, 'total_flos': 1.1631478161014784e+16, 'train_loss': 0.43342361098020155, 'epoch': 3.0})

In [ ]:
# Evaluate the model on the test set
results = trainer.evaluate(tokenized_datasets["test"])
print("Test Results:", results)

Test Results: {'eval_loss': 0.40019485354423523, 'eval_runtime': 19.6834, 'eval_samples_per_second': 322.86, 'eval_steps_per_second': 20.22, 'epoch': 3.0}


In [ ]:
# Save the model and tokenizer to your Google Drive
model.save_pretrained("/content/drive/MyDrive/models/fr-en-translation_model")
tokenizer.save_pretrained("/content/drive/MyDrive/models/fr-en-translation_model")

('/content/drive/MyDrive/models/fr-en-translation_model/tokenizer_config.json',
 '/content/drive/MyDrive/models/fr-en-translation_model/special_tokens_map.json',
 '/content/drive/MyDrive/models/fr-en-translation_model/vocab.json',
 '/content/drive/MyDrive/models/fr-en-translation_model/source.spm',
 '/content/drive/MyDrive/models/fr-en-translation_model/target.spm',
 '/content/drive/MyDrive/models/fr-en-translation_model/added_tokens.json')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the model and tokenizer from your saved path
model_path = "/content/drive/MyDrive/models/fr-en-translation_model"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [ ]:
def translate(sentence, model, tokenizer, max_length=50):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True)

    # Generate translation using the model
    outputs = model.generate(**inputs, max_length=max_length, num_beams=5, early_stopping=True)

    # Decode the output tokens to text
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation


In [ ]:
# Example sentences to test
test_sentences = [
"Bonjour, comment ça va ?"
"J'aime apprendre de nouvelles langues."
"La technologie change rapidement le monde."
"Ce livre est très intéressant et éducatif."
"Peux-tu m'aider avec ce problème ?"
]

# Generate translations
for sentence in test_sentences:
    english_translation = translate(sentence, model, tokenizer)
    print(f"French: {sentence}")
    print(f"English: {english_translation}\n")


French: Bonjour, comment ça va ?J'aime apprendre de nouvelles langues.La technologie change rapidement le monde.Ce livre est très intéressant et éducatif.Peux-tu m'aider avec ce problème ?
English: How are things? I like learning new languages, and the world is rapidly changed by the technology. This book is very interesting and instructive. Can you help me with this problem?"

